In [1]:
# import necessary imports
import numpy as np
#from numpy_ml.neural_nets.optimizers import Adam
import torch
from torch import nn, optim

In [68]:
def get_batch(input, batch_size,chunk_size):

    input_batch = []
    # print(type(input_batch))
    target_batch = []
    idx = np.random.randint(0,len(input)-(chunk_size+1),size=batch_size)
    for i in range(0,len(idx)-1):
        input_batch.append(input[idx[i]:idx[i]+chunk_size])
        target_batch.append(input[idx[i]+1:idx[i]+(chunk_size+1)])
    
    input_batch = np.array(input_batch)
    target_batch = np.array(target_batch)

    return input_batch, target_batch

In [69]:
with open (r"Shakespeare_byte.txt", 'r') as f:
   shakespeare_byte_train = eval(f.read())

with open (r"vocab_train.txt", 'r') as f:
   vocab_train = eval(f.read())

vocab = vocab_train
indices = np.arange(0,len(vocab),1)
inidces = indices.astype(int)
indices = indices.tolist()
key_byte = dict(zip(vocab, indices))
value_byte = dict(zip(indices,vocab))

# Map each token in shakespeare_byte_train to its index using key_byte
indices_translation = [key_byte[token] for token in shakespeare_byte_train if token in key_byte]

with open('indices_text.txt', 'w') as indices_text:
    indices_text.write(str(indices_translation))

with open (r"indices_text.txt", 'r') as f:
  indices_text = eval(f.read())


bytes_translation = [value_byte[token] for token in indices_text if token in value_byte]

with open('bytes_text.txt', 'w') as bytes_text:
    bytes_text.write(str(bytes_translation))


In [70]:
with open (r"indices_text.txt", 'r') as f:
  indices_text = eval(f.read())
  
x,y = get_batch(indices_text,4,8)
print(x.shape)
print(x)
print(y.shape)
print(y)

(3, 8)
[[1309 1154  176 1219 1263  437 1499 1022]
 [1264 1262 1196 1267  840 1401 1425  794]
 [1251 1447 1221 1539 1295  457  110  452]]
(3, 8)
[[1154  176 1219 1263  437 1499 1022 1260]
 [1262 1196 1267  840 1401 1425  794 1262]
 [1447 1221 1539 1295  457  110  452  110]]


In [71]:
def decode_characters(input):
    """Decodes a list of indices back to their corresponding characters
    given the abive defined vocabulary"""

    decoded = [] #given the input, we will decode it back to characters
    for i in range(0,len(input)):
        decoded.append(value_byte[input[i]])#using the translation dctionary: value_byte
#make its prettier by joining list to actual words and replacing underscores with spaces
    decoded = ''.join(decoded)
    decoded = decoded.replace('_', ' ')
    return decoded

In [64]:
class neural_embedding:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.token_embedding_table = np.random.rand(vocab_size, vocab_size)

    def calculate_softmax(self, x):
        """Takes input array x and returns softmax."""
        soft_x = np.exp(x - np.max(x))
        softer_x = soft_x / np.sum(soft_x)
        return softer_x

    def calculate_cross_entropy(self, y_hatless, y_hat):
        """
        Takes target (y_hatless) and prediction (y_hat) and computes cross entropy loss.
        """
        # get vocab_size
        _, _, vocab_size = y_hat.shape        
        y_hat = y_hat.reshape(y_hat.shape[0]*y_hat.shape[1], y_hat.shape[2])
        y_hatless_flat = y_hatless.reshape(-1)
        # one-hot encode targets
        y_hatless_hot = np.eye(vocab_size)[y_hatless_flat]
       
        y_hat = self.calculate_softmax(y_hat)
    
        return -np.sum(y_hatless_hot*np.log(y_hat))
    
    def forward(self, idx, targets=None):
        """
        Implements forward pass with an unnecessary logitte function 
        which i only did not delete because now I'm emotionally attached.
        Args:
            idx(np.array): (B,T) numpy array of integers
            targets(np.array): (B,T) numpy array of integers
        Returns:
            input_logits(np.array)
            sometimes also: targets(np.array)
        """
        batch_size, chunk_size = idx.shape
        logits = np.zeros((batch_size, chunk_size, (self.token_embedding_table[0].size)))

        def logitte(batch_size, chunk_size, input):
            for batch in range(batch_size):
                for chunk in range(chunk_size):
                    # (B,T,C) b=batch_size, t="time"=chunk_size, c=vocab_size
                    logits[batch][chunk] = self.token_embedding_table[input[batch][chunk]]
                    
            return logits

        input_logits = logitte(batch_size, chunk_size, idx)
        
        if targets is not None:
            loss = self.calculate_cross_entropy(targets, input_logits)

            return input_logits, loss

        return input_logits

    
    def backward(self, targets, input_logits):
        # need to do the same reshaping as we did for cross entropy, apparently
        targets_flat = targets.reshape(-1)
        one_hot_targets = np.eye(self.vocab_size)[targets_flat]

        # shape after: ((batch_size*chunk_size), vocab_size)
        input_logits_2d = input_logits.reshape(input_logits.shape[0]*input_logits.shape[1], input_logits.shape[2])

        # somehow this is supposedly the combiantion of the derivative of softmax with the derivative of the CCE
        delta = one_hot_targets - input_logits_2d

        # want shape (80,80) for matrix multiplication, but with correct indices (use one-hot targets for that)
        delta_indexed = np.dot(one_hot_targets.transpose(),delta)

        # compute gradient for weight matrix: dot product between the transpose of the to layer and delta vector computed above
        gradient = (self.token_embedding_table @ delta_indexed) 

    
        return gradient
    
    
    def generate(self,idx,max_new_tokens):
        collect = idx.tolist()
        for _ in range(max_new_tokens):
            logits = self.forward(idx)
            logits = logits[:, -1, :]  # take the last time step
            probs = self.calculate_softmax(logits)
            idx_next = np.random.multinomial(1, probs[0])
            print(probs[0][:10])
            idx_next = np.where(idx_next==1)
            idx[0][:-1] = idx[0][1:]  # shift the array to the left
            idx[0][-1] = idx_next[0]
            collect[0].append(idx_next[0][0].tolist())


        return collect


    

In [17]:
# currently do not use loss at all, so something is probably very wrong

def train(model, text, optimiser, token_embedding_table, train_step):
    batch_size=2
    chunk_size = 8
    for steps in range(train_step): # TODO: please increase
        # sample batch of data
        xb, yb = get_batch(text, batch_size, chunk_size) # TODO: pls adapt to above fct
        optimiser.zero_grad()  # reset gradients
        # get logits and loss
        logits, loss = model.forward(xb, yb)
        print(f"loss: {loss}")

        gradient = model.backward(yb, logits)
        token_embedding_table.grad = torch.tensor(gradient, dtype=torch.float32)
        optimiser.step()  # apply gradients to parameters
        # diff = model.token_embedding_table - np.array(token_embedding_table)
        # print(f"Difference in weights before - afte Adam: {diff}")
        model.token_embedding_table = np.array(token_embedding_table)
        

In [18]:
with open (r"indices_text.txt", 'r') as f:
    indices_text = eval(f.read())
with open(r"vocab_train.txt", 'r') as f:
    vocab_train = eval(f.read())

In [76]:
vocab_size = len(vocab_train)

my_neural_embedding = neural_embedding(vocab_size)
param_dict = {"weight": my_neural_embedding.token_embedding_table} # one entry, key is weight and value is my_neural

tensor = torch.tensor(my_neural_embedding.token_embedding_table, dtype=torch.float32)
optimiser = optim.Adam([tensor], lr=0.05)
train(my_neural_embedding, indices_text, optimiser, tensor, 10) 
our_output = my_neural_embedding.generate(np.array([[27,7,20,3,666]]), max_new_tokens=10)

decoded = decode_characters(our_output[0])
print(decoded)

 # example usage of generate method


loss: 74.8841135208031
loss: 73.90905389118407
loss: 77.08283622393236


C:\Users\belan\AppData\Local\Temp\ipykernel_17832\132296554.py:19: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  model.token_embedding_table = np.array(token_embedding_table)


loss: 74.6184399737403
loss: 75.33619550185036
loss: 76.46228423951331
loss: 75.37052029241359
loss: 75.8539667224812
loss: 76.4819888354655
loss: 75.47418249705825
[0.00060778 0.00039001 0.00059341 0.00090653 0.00057909 0.00054772
 0.00099646 0.00053803 0.00039882 0.00040291]
[0.00055205 0.00085624 0.00044872 0.00042573 0.00062704 0.00060983
 0.00040216 0.0004451  0.00065721 0.0008329 ]
[0.00101701 0.00099932 0.00038361 0.00084106 0.00054851 0.00067634
 0.00084708 0.00043439 0.00059267 0.00037695]
[0.00082889 0.00052736 0.00070014 0.00094304 0.00046292 0.00099572
 0.0006982  0.00046387 0.00061094 0.00055781]
[0.00087146 0.00043275 0.00049824 0.0004156  0.00060558 0.00070125
 0.00067006 0.00073844 0.00082479 0.00078393]
[0.00100984 0.00083845 0.00098293 0.00047278 0.00053314 0.00053373
 0.00045362 0.0005239  0.00100085 0.00067128]
[0.00048743 0.00061111 0.00054496 0.00045846 0.00055329 0.00043714
 0.00062214 0.00093085 0.00069166 0.00068655]
[0.00040566 0.00063968 0.00099095 0.00077728

C:\Users\belan\AppData\Local\Temp\ipykernel_17832\3722754344.py:90: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  idx[0][-1] = idx_next[0]
